In [1]:
import os
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm 

import functools as ft
from collections import defaultdict

import seaborn as sns
import matplotlib.pyplot as plt

In [43]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 60)

In [3]:
pd.options.mode.chained_assignment = None

# ED Visit Rates

In [8]:
mmd_data = pd.read_csv('mmd_data.csv')
mmd_data

,population,year,geography,measure,adjustment,analysis,domain,condition,primary_sex,primary_age,primary_dual,fips,county,state,urban,primary_race,primary_eligibility,primary_denominator,analysis_value
0,Medicare Fee For Service,2022,County,Emergency department visit rate,Unsmoothed actual,Base measure,Primary chronic conditions,All Emergency Department Visits,All,All,Dual & non-dual,1001,Autauga County,ALABAMA,Urban,All,All,undefined,588
1,Medicare Fee For Service,2022,County,Emergency department visit rate,Unsmoothed actual,Base measure,Primary chronic conditions,All Emergency Department Visits,All,All,Dual & non-dual,1003,Baldwin County,ALABAMA,Rural,All,All,undefined,608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3216,Medicare Fee For Service,2022,County,Emergency department visit rate,Unsmoothed actual,Base measure,Primary chronic conditions,All Emergency Department Visits,All,All,Dual & non-dual,72151,Yabucoa Municipio,PUERTO RICO,Urban,All,All,undefined,460
3217,Medicare Fee For Service,2022,County,Emergency department visit rate,Unsmoothed actual,Base measure,Primary chronic conditions,All Emergency Department Visits,All,All,Dual & non-dual,72153,Yauco Municipio,PUERTO RICO,Urban,All,All,undefined,528


# County Health Data

In [9]:
df_CHR = pd.read_excel('2022 County Health Rankings Data - v1.xlsx', header=[0, 1], sheet_name='Ranked Measure Data')
df_CHR.head(3)

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0 Premature death  \
                FIPS              State             County      Unreliable   
0               1000            Alabama                NaN             NaN   
1               1001            Alabama            Autauga             NaN   
2               1003            Alabama            Baldwin             NaN   

                                                                          \
    Deaths Years of Potential Life Lost Rate  95% CI - Low 95% CI - High   
0  88086.0                      10350.071456  10246.133257  10454.009654   
1    836.0                       8027.394727   7197.625811   8857.163642   
2   3377.0                       8118.358206   7666.874101   8569.842311   

                                                           \
  Quartile YPLL Rate (AIAN) YPLL Rate (AIAN) 95% CI - Low   
0      NaN      5967.152772                   4839.918333   
1      1.0              NaN                           NaN   
2      1.0              NaN                           NaN   

                                                              \
  YPLL Rate (AIAN) 95% CI - High YPLL Rate (AIAN) Unreliable   
0                    7094.387212                         NaN   
1                            NaN                         NaN   
2                            NaN                         NaN   

                                                    \
  YPLL Rate (Asian) YPLL Rate (Asian) 95% CI - Low   
0       3410.772412                    2945.367818   
1               NaN                            NaN   
2               NaN                            NaN   

                                                                \
  YPLL Rate (Asian) 95% CI - High YPLL Rate (Asian) Unreliable   
0                     3876.177007                          NaN   
1                             NaN                          NaN   
2                             NaN                          NaN   

                                                    \
  YPLL Rate (Black) YPLL Rate (Black) 95% CI - Low   
0      13245.058385                   13022.778798   
1      11548.535678                    9369.191471   
2      11603.143788                    9970.802485   

                                                                \
  YPLL Rate (Black) 95% CI - High YPLL Rate (Black) Unreliable   
0                    13467.337973                          NaN   
1                    13727.879885                          NaN   
2                    13235.485090                          NaN   

                                                          \
  YPLL Rate (Hispanic) YPLL Rate (Hispanic) 95% CI - Low   
0          5244.424847                       4900.917319   
1                  NaN                               NaN   
2          4590.548042                       3232.169463   

                                                                      \
  YPLL Rate (Hispanic) 95% CI - High YPLL Rate (Hispanic) Unreliable   
0                        5587.932376                             NaN   
1                                NaN                             NaN   
2                        6327.465781                             NaN   

                                                    \
  YPLL Rate (white) YPLL Rate (white) 95% CI - Low   
0       9562.516718                    9438.715332   
1       7332.553866                    6411.306591   
2       8058.609145                    7555.295103   

                                                                \
  YPLL Rate (white) 95% CI - High YPLL Rate (white) Unreliable   
0                     9686.318103                          NaN   
1                     8253.801142                          NaN   
2                     8561.923188                          NaN   

    Poor or fair health                                      \
  % Fair or Poor Health 95% CI - Low 95% CI - High Quartile   
0        

In [10]:
# The next two cells cleans up the two header issue from the county health rankings dataset.
geography = ['Unnamed: 0_level_0','Unnamed: 1_level_0','Unnamed: 2_level_0']
sections = ['Primary care physicians','Dentists','Mental health providers']

In [11]:
df_CHR_Features = df_CHR.iloc[:, df_CHR.columns.isin(geography+sections, level=0)]
df_CHR_Features.columns = [col[1] if col[0].startswith('Unnamed:') else ': '.join(col) for col in df_CHR_Features.columns.values]
df_CHR_Features['FIPS'] = df_CHR_Features['FIPS'].apply(str).str.zfill(5)
df_CHR_Features.head(1)

,FIPS,State,County,Primary care physicians: # Primary Care Physicians,Primary care physicians: Primary Care Physicians Rate,Primary care physicians: Primary Care Physicians Ratio,Primary care physicians: Quartile,Dentists: # Dentists,Dentists: Dentist Rate,Dentists: Dentist Ratio,Dentists: Quartile,Mental health providers: # Mental Health Providers,Mental health providers: Mental Health Provider Rate,Mental health providers: Mental Health Provider Ratio,Mental health providers: Quartile
0,01000,Alabama,NaN,3228.0,65.83476,1519:1,NaN,2429.0,49.35455,2026:1,NaN,5818.0,118.21522,846:1,NaN


In [12]:
mmd_data['fips'] = mmd_data.fips.apply(str).str.zfill(5)
mmd_data.head()

,population,year,geography,measure,adjustment,analysis,domain,condition,primary_sex,primary_age,primary_dual,fips,county,state,urban,primary_race,primary_eligibility,primary_denominator,analysis_value
0,Medicare Fee For Service,2022,County,Emergency department visit rate,Unsmoothed actual,Base measure,Primary chronic conditions,All Emergency Department Visits,All,All,Dual & non-dual,01001,Autauga County,ALABAMA,Urban,All,All,undefined,588
1,Medicare Fee For Service,2022,County,Emergency department visit rate,Unsmoothed actual,Base measure,Primary chronic conditions,All Emergency Department Visits,All,All,Dual & non-dual,01003,Baldwin County,ALABAMA,Rural,All,All,undefined,608
2,Medicare Fee For Service,2022,County,Emergency department visit rate,Unsmoothed actual,Base measure,Primary chronic conditions,All Emergency Department Visits,All,All,Dual & non-dual,01005,Barbour County,ALABAMA,Rural,All,All,undefined,747
3,Medicare Fee For Service,2022,County,Emergency department visit rate,Unsmoothed actual,Base measure,Primary chronic conditions,All Emergency Department Visits,All,All,Dual & non-dual,01007,Bibb County,ALABAMA,Urban,All,All,undefined,593
4,Medicare Fee For Service,2022,County,Emergency department visit rate,Unsmoothed actual,Base measure,Primary chronic conditions,All Emergency Department Visits,All,All,Dual & non-dual,01009,Blount County,ALABAMA,Urban,All,All,undefined,601


In [13]:
merge_df = mmd_data.merge(df_CHR_Features,how='left',left_on='fips',right_on='FIPS')

In [14]:
merge_df = merge_df.drop(['FIPS','State','County'],axis=1)

In [15]:
merge_df = merge_df.drop(labels=['population','year','geography','measure','adjustment','analysis','domain','condition','primary_sex'\
                         ,'primary_age','primary_dual','primary_race','primary_eligibility','primary_denominator'],axis=1)


In [16]:
merge_df = merge_df.drop(labels=['Primary care physicians: # Primary Care Physicians','Dentists: # Dentists','Mental health providers: # Mental Health Providers','Primary care physicians: Quartile','Dentists: Quartile','Mental health providers: Quartile'],axis=1)

In [18]:
for i in merge_df.columns:
    if('ratio' in i.lower()):
        merge_df[i]=merge_df[i].str[:-2].astype(float)


In [19]:
HPSA_DH = pd.read_csv('HRSA/BCD_HPSA_FCT_DET_DH.csv')
HPSA_DH['HPSA Score'].value_counts()
HPSA_DH['State and County Federal Information Processing Standard Code']
HPSA_DH.columns

0     8208
19    4883
      ... 
2       25
1       20
Name: HPSA Score, Length: 27, dtype: int64

0        54039
1        54039
         ...  
37946    06023
37947    06023
Name: State and County Federal Information Processing Standard Code, Length: 37948, dtype: object

Index(['HPSA Name', 'HPSA ID', 'Designation Type', 'HPSA Discipline Class',
       'HPSA Score', 'Primary State Abbreviation', 'HPSA Status',
       'HPSA Designation Date', 'HPSA Designation Last Update Date',
       'Metropolitan Indicator', 'HPSA Geography Identification Number',
       'HPSA Degree of Shortage', 'Withdrawn Date', 'HPSA FTE',
       'HPSA Designation Population', '% of Population Below 100% Poverty',
       'HPSA Formal Ratio', 'HPSA Population Type', 'Rural Status',
       'Longitude', 'Latitude', 'BHCMIS Organization Identification Number',
       'Break in Designation', 'Common County Name', 'Common Postal Code',
       'Common Region Name', 'Common State Abbreviation',
       'Common State County FIPS Code', 'Common State FIPS Code',
       'Common State Name', 'County Equivalent Name',
       'County or County Equivalent Federal Information Processing Standard Code',
       'Discipline Class Number', 'HPSA Address', 'HPSA City',
       'HPSA Component Name', 'H

In [20]:
def condense_df(df,prefix):
    df_subset = df[['HPSA Score','% of Population Below 100% Poverty','Common State County FIPS Code','HPSA Estimated Underserved Population','HPSA Estimated Served Population']]
    grouped_df = df_subset.groupby('Common State County FIPS Code').agg('mean')
    
    if('XXXXX' in grouped_df.index):
        grouped_df.drop(['XXXXX'],inplace=True)
    
    mapping = {}
    for column in df_subset.columns:
        mapping[column]=prefix+column
    grouped_df = grouped_df.rename(mapper=mapping,axis='columns')
    
    return grouped_df

HPSA_DH_SUB = condense_df(HPSA_DH,"Dental Health: ")
HPSA_DH_SUB['FIPS']=HPSA_DH_SUB.index
HPSA_DH_SUB

,Dental Health: HPSA Score,Dental Health: % of Population Below 100% Poverty,Dental Health: HPSA Estimated Underserved Population,Dental Health: HPSA Estimated Served Population,FIPS
Common State County FIPS Code,,,,,
01001,17.0,16.900000,65590.0,9556.000000,01001
01003,8.5,17.900000,182060.0,49960.000000,01003
...,...,...,...,...,...
78020,12.0,26.033333,1331.0,10666.666667,78020
78030,14.5,24.200000,12687.0,30966.666667,78030


In [21]:
df1 = merge_df.merge(HPSA_DH_SUB,how='left',left_on='fips',right_on='FIPS')
df1.drop(labels=['FIPS'],axis=1,inplace=True)
df1.isna().sum()

fips                                                      0
county                                                   13
                                                       ... 
Dental Health: HPSA Estimated Underserved Population    471
Dental Health: HPSA Estimated Served Population         471
Length: 15, dtype: int64

In [22]:
HPSA_MH = pd.read_csv('HRSA/BCD_HPSA_FCT_DET_MH.csv')
HPSA_MH_SUB = condense_df(HPSA_MH,'Mental Health: ')
HPSA_MH_SUB['FIPS']=HPSA_MH_SUB.index
HPSA_MH_SUB

,Mental Health: HPSA Score,Mental Health: % of Population Below 100% Poverty,Mental Health: HPSA Estimated Underserved Population,Mental Health: HPSA Estimated Served Population,FIPS
Common State County FIPS Code,,,,,
01001,13.000000,18.050000,68234.000000,181525.000000,01001
01003,13.250000,12.633333,26996.545455,15288.636364,01003
...,...,...,...,...,...
78020,6.666667,33.700000,4100.000000,0.000000,78020
78030,12.500000,29.133333,13547.000000,36830.000000,78030


In [23]:
df2 = df1.merge(HPSA_MH_SUB,how='left',left_on='fips',right_on='FIPS')
df2.drop(labels=['FIPS'],axis=1,inplace=True)
df2.isna().sum()
df2

fips                                                      0
county                                                   13
                                                       ... 
Mental Health: HPSA Estimated Underserved Population    228
Mental Health: HPSA Estimated Served Population         228
Length: 19, dtype: int64

,fips,county,state,urban,analysis_value,Primary care physicians: Primary Care Physicians Rate,Primary care physicians: Primary Care Physicians Ratio,Dentists: Dentist Rate,Dentists: Dentist Ratio,Mental health providers: Mental Health Provider Rate,Mental health providers: Mental Health Provider Ratio,Dental Health: HPSA Score,Dental Health: % of Population Below 100% Poverty,Dental Health: HPSA Estimated Underserved Population,Dental Health: HPSA Estimated Served Population,Mental Health: HPSA Score,Mental Health: % of Population Below 100% Poverty,Mental Health: HPSA Estimated Underserved Population,Mental Health: HPSA Estimated Served Population
0,01001,Autauga County,ALABAMA,Urban,588,44.74753,2235.0,33.84095,2955.0,37.40315,2674.0,17.0,16.9,65590.0,9556.0,13.000000,18.050000,68234.000000,181525.000000
1,01003,Baldwin County,ALABAMA,Rural,608,68.98591,1450.0,48.84708,2047.0,99.43869,1006.0,8.5,17.9,182060.0,49960.0,13.250000,12.633333,26996.545455,15288.636364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3216,72151,Yabucoa Municipio,PUERTO RICO,Urban,460,NaN,NaN,NaN,NaN,NaN,NaN,12.0,51.6,31808.0,68491.0,10.333333,52.550000,55490.500000,10000.000000
3217,72153,Yauco Municipio,PUERTO RICO,Urban,528,NaN,NaN,NaN,NaN,NaN,NaN,12.0,48.8,65217.5,15888.0,10.500000,49.000000,29424.000000,0.000000


In [24]:
HPSA_PC = pd.read_csv('HRSA/BCD_HPSA_FCT_DET_PC.csv')
HPSA_PC_SUB = condense_df(HPSA_PC,'Primary Care: ')
HPSA_PC_SUB['FIPS']=HPSA_PC_SUB.index
HPSA_PC_SUB

,Primary Care: HPSA Score,Primary Care: % of Population Below 100% Poverty,Primary Care: HPSA Estimated Underserved Population,Primary Care: HPSA Estimated Served Population,FIPS
Common State County FIPS Code,,,,,
01001,13.000000,19.126667,42831.866667,40162.333333,01001
01003,13.666667,15.011538,16227.230769,19090.192308,01003
...,...,...,...,...,...
78020,15.500000,27.350000,4905.750000,5087.500000,78020
78030,8.428571,25.720000,-2481.500000,24325.000000,78030


In [25]:
df3 = df2.merge(HPSA_PC_SUB,how='left',left_on='fips',right_on='FIPS')
df3.drop(labels=['FIPS'],axis=1,inplace=True)
df3.isna().sum()
df3

fips                                                     0
county                                                  13
                                                      ... 
Primary Care: HPSA Estimated Underserved Population    152
Primary Care: HPSA Estimated Served Population         152
Length: 23, dtype: int64

,fips,county,state,urban,analysis_value,Primary care physicians: Primary Care Physicians Rate,Primary care physicians: Primary Care Physicians Ratio,Dentists: Dentist Rate,Dentists: Dentist Ratio,Mental health providers: Mental Health Provider Rate,Mental health providers: Mental Health Provider Ratio,Dental Health: HPSA Score,Dental Health: % of Population Below 100% Poverty,Dental Health: HPSA Estimated Underserved Population,Dental Health: HPSA Estimated Served Population,Mental Health: HPSA Score,Mental Health: % of Population Below 100% Poverty,Mental Health: HPSA Estimated Underserved Population,Mental Health: HPSA Estimated Served Population,Primary Care: HPSA Score,Primary Care: % of Population Below 100% Poverty,Primary Care: HPSA Estimated Underserved Population,Primary Care: HPSA Estimated Served Population
0,01001,Autauga County,ALABAMA,Urban,588,44.74753,2235.0,33.84095,2955.0,37.40315,2674.0,17.0,16.9,65590.0,9556.0,13.000000,18.050000,68234.000000,181525.000000,13.000000,19.126667,42831.866667,40162.333333
1,01003,Baldwin County,ALABAMA,Rural,608,68.98591,1450.0,48.84708,2047.0,99.43869,1006.0,8.5,17.9,182060.0,49960.0,13.250000,12.633333,26996.545455,15288.636364,13.666667,15.011538,16227.230769,19090.192308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3216,72151,Yabucoa Municipio,PUERTO RICO,Urban,460,NaN,NaN,NaN,NaN,NaN,NaN,12.0,51.6,31808.0,68491.0,10.333333,52.550000,55490.500000,10000.000000,6.000000,100.000000,86040.666667,27000.000000
3217,72153,Yauco Municipio,PUERTO RICO,Urban,528,NaN,NaN,NaN,NaN,NaN,NaN,12.0,48.8,65217.5,15888.0,10.500000,49.000000,29424.000000,0.000000,4.250000,76.350000,58350.750000,30495.000000


In [24]:
df3.isna().sum()

fips                                                        0
county                                                     13
state                                                       0
urban                                                       0
analysis_value                                              0
Primary care physicians: Primary Care Physicians Rate     240
Primary care physicians: Primary Care Physicians Ratio    240
Dentists: Dentist Rate                                    180
Dentists: Dentist Ratio                                   180
Mental health providers: Mental Health Provider Rate      294
Mental health providers: Mental Health Provider Ratio     294
Dental Health: HPSA Score                                 240
Dental Health: % of Population Below 100% Poverty         601
Dental Health: HPSA Estimated Underserved Population      471
Dental Health: HPSA Estimated Served Population           471
Mental Health: HPSA Score                                  56
Mental H

In [28]:
df3.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
analysis_value,3218.0,592.767557,114.948989,0.0,523.000000,592.000000,664.000000,1.167000e+03
Primary care physicians: Primary Care Physicians Rate,2978.0,54.444955,36.456786,0.0,30.785488,47.164885,71.353590,5.869446e+02
Primary care physicians: Primary Care Physicians Ratio,2978.0,2707.325386,2373.255195,169.0,1358.250000,2052.500000,3076.000000,2.832400e+04
Dentists: Dentist Rate,3038.0,47.427330,33.138715,0.0,27.114980,42.777830,63.218740,7.487442e+02
Dentists: Dentist Ratio,3038.0,2917.436142,2425.496528,134.0,1527.250000,2228.500000,3340.000000,3.045700e+04
Mental health providers: Mental Health Provider Rate,2924.0,179.831302,196.975079,0.0,54.392178,123.934290,237.341862,2.764911e+03
Mental health providers: Mental Health Provider Ratio,2924.0,1574.484952,2282.013955,36.0,420.000000,789.000000,1730.250000,3.286500e+04
Dental Health: HPSA Score,2978.0,12.701725,5.081953,0.0,10.000000,13.447632,16.200000,2.600000e+01
Dental Health: % of Population Below 100% Poverty,2617.0,18.149467,8.753537,0.0,12.870455,16.400000,21.300000,8.078571e+01
Dental Health: HPSA Estimated Underserved Population,2747.0,10922.090451,17876.606274,-130243.0,2398.625000,5975.048780,13951.438073,1.820600e+05


In [37]:
df3.shape

(3218, 23)

In [38]:
df3

,fips,county,state,urban,analysis_value,Primary care physicians: Primary Care Physicians Rate,Primary care physicians: Primary Care Physicians Ratio,Dentists: Dentist Rate,Dentists: Dentist Ratio,Mental health providers: Mental Health Provider Rate,Mental health providers: Mental Health Provider Ratio,Dental Health: HPSA Score,Dental Health: % of Population Below 100% Poverty,Dental Health: HPSA Estimated Underserved Population,Dental Health: HPSA Estimated Served Population,Mental Health: HPSA Score,Mental Health: % of Population Below 100% Poverty,Mental Health: HPSA Estimated Underserved Population,Mental Health: HPSA Estimated Served Population,Primary Care: HPSA Score,Primary Care: % of Population Below 100% Poverty,Primary Care: HPSA Estimated Underserved Population,Primary Care: HPSA Estimated Served Population
0,01001,Autauga County,ALABAMA,Urban,588,44.74753,2235.0,33.84095,2955.0,37.40315,2674.0,17.00,16.90,65590.0,9556.0,13.000000,18.050000,68234.000000,181525.000000,13.000000,19.126667,42831.866667,40162.333333
1,01003,Baldwin County,ALABAMA,Rural,608,68.98591,1450.0,48.84708,2047.0,99.43869,1006.0,8.50,17.90,182060.0,49960.0,13.250000,12.633333,26996.545455,15288.636364,13.666667,15.011538,16227.230769,19090.192308
2,01005,Barbour County,ALABAMA,Rural,747,36.45791,2743.0,36.60173,2732.0,24.40115,4098.0,16.25,24.40,11858.5,3260.0,14.200000,19.400000,41980.000000,112400.000000,10.888889,29.900000,-139.000000,10305.833333
3,01007,Bibb County,ALABAMA,Urban,593,58.05126,1723.0,27.10517,3689.0,27.10517,3689.0,15.80,18.15,32300.5,12598.5,16.800000,18.500000,86303.500000,3975.000000,16.750000,18.100000,2607.000000,18025.000000
4,01009,Blount County,ALABAMA,Urban,601,22.48124,4448.0,19.00517,5262.0,17.27742,5788.0,15.50,15.45,17867.0,3586.5,18.500000,16.850000,234168.500000,48520.000000,7.000000,13.500000,4170.000000,52938.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3213,72145,Vega Baja Municipio,PUERTO RICO,Urban,436,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,38534.0,48000.0,10.000000,51.800000,233700.000000,8600.000000,0.000000,NaN,43034.000000,43500.000000
3214,72147,Vieques Municipio,PUERTO RICO,Rural,304,NaN,NaN,NaN,NaN,NaN,NaN,13.00,52.60,24057.0,70120.0,12.333333,47.050000,8069.000000,0.000000,11.500000,52.600000,23926.000000,69520.000000
3215,72149,Villalba Municipio,PUERTO RICO,Urban,392,NaN,NaN,NaN,NaN,NaN,NaN,9.00,46.40,31127.5,50256.0,13.000000,48.550000,50348.500000,11300.000000,4.666667,73.900000,36449.666667,50707.666667
3216,72151,Yabucoa Municipio,PUERTO RICO,Urban,460,NaN,NaN,NaN,NaN,NaN,NaN,12.00,51.60,31808.0,68491.0,10.333333,52.550000,55490.500000,10000.000000,6.000000,100.000000,86040.666667,27000.000000


# AHRF Data

In [26]:
with open('HRSA/AHRF/ahrf2023GEO.csv') as f:
    print(f)

<_io.TextIOWrapper name='HRSA/AHRF/ahrf2023GEO.csv' mode='r' encoding='cp1252'>


In [27]:
#HP, HF
AHRF_HF = pd.read_csv('HRSA/AHRF/ahrf2023HF.csv')
AHRF_HF.fips_st_cnty = AHRF_HF.fips_st_cnty.apply(str).str.zfill(5)
AHRF_HF

,fips_st_cnty,cnty_name_st_abbrev,hosp_21,hosp_20,stgh_21,stgh_20,stngh_21,stngh_20,lth_21,lth_20,sth_psych_21,sth_psych_20,sth_rehab_21,sth_rehab_20,sth_child_med_21,sth_child_med_20,sth_child_psych_21,sth_child_psych_20,stlt_acute_cares_21,stlt_acute_cares_20,ltgh_med_surg_hosp_21,ltgh_med_surg_hosp_20,lth_psych_21,lth_psych_20,lth_rehab_21,lth_rehab_20,lth_chronc_dis_21,lth_chronc_dis_20,lth_intel_disblts_21,lth_intel_disblts_20,lth_child_psych_21,lth_child_psych_20,lth_acute_long_care_hosp_21,lth_acute_long_care_hosp_20,sth_comn_21,sth_comn_20,vetn_hosp_21,vetn_hosp_20,critcl_access_hosp_21,critcl_access_hosp_20,stgh_critcl_access_21,stgh_critcl_access_20,rural_refrrl_ctr_21,rural_refrrl_ctr_20,sole_comn_provdrs_21,sole_comn_provdrs_20,stgh_jcaho_accred_21,stgh_jcaho_accred_20,stnglth_jcaho_accred_21,stnglth_jcaho_accred_20,stgh_dnv_accred_21,stgh_dnv_accred_20,stnglth_dnv_accred_21,stnglth_dnv_accred_20,stgh_acs_cancr_pgm_approvd_21,stgh_acs_cancr_pgm_approvd_20,stnglth_acs_cancr_pgm_approvd_21,stnglth_acs_cancr_pgm_approvd_20,stgh_resdncy_tranng_21,stgh_resdncy_tranng_20,stnglth_resdncy_tranng_21,stnglth_resdncy_tranng_20,stgh_med_sch_affil_21,stgh_med_sch_affil_20,stnglth_med_sch_affil_21,stnglth_med_sch_affil_20,vetn_hosp_med_sch_affil_21,vetn_hosp_med_sch_affil_20,stgh_carf_accred_21,stgh_carf_accred_20,stnglth_carf_accred_21,stnglth_carf_accred_20,stgh_membr_coth_21,stgh_membr_coth_20,vetn_hosp_membr_coth_21,vetn_hosp_membr_coth_20,sth_comn_membr_coth_21,sth_comn_membr_coth_20,stnglth_membr_coth_21,stnglth_membr_coth_20,stgh_aoa_accred_21,stgh_aoa_accred_20,stnglth_aoa_accred_21,stnglth_aoa_accred_20,stgh_medcr_cert_21,stgh_medcr_cert_20,stnglth_medcr_cert_21,stnglth_medcr_cert_20,hosp_adm_21,hosp_adm_20,stgh_hosp_adm_21,stgh_hosp_adm_20,stnglth_hosp_adm_21,stnglth_hosp_adm_20,sth_comn_hosp_adm_21,sth_comn_hosp_adm_20,vetn_hosp_hosp_adm_21,vetn_hosp_hosp_adm_20,hosp_nh_adm_21,hosp_nh_adm_20,stgh_nh_adm_21,stgh_nh_adm_20,stnglth_nh_adm_21,stnglth_nh_adm_20,medcr_ffs_acute_readm_21,medcr_ffs_acute_readm_20,medcr_ffs_hosp_readm_rate_21,medcr_ffs_hosp_readm_rate_20,medcr_ffs_prev_hosp_rate_20,medcr_ffs_prev_hosp_rate_19,hosp_beds_21,hosp_beds_20,stgh_hosp_beds_21,stgh_hosp_beds_20,stngh_hosp_beds_21,stngh_hosp_beds_20,lth_hosp_beds_21,lth_hosp_beds_20,sth_comn_hosp_beds_21,sth_comn_hosp_beds_20,vetn_hosp_hosp_beds_21,vetn_hosp_hosp_beds_20,hosp_nh_beds_21,hosp_nh_beds_20,stgh_nh_beds_21,stgh_nh_beds_20,stngh_nh_beds_21,stngh_nh_beds_20,lth_nh_beds_21,lth_nh_beds_20,hosp_licd_beds_21,hosp_licd_beds_20,stgh_licd_hosp_beds_21,stgh_licd_hosp_beds_20,stngh_licd_hosp_beds_21,stngh_licd_hosp_beds_20,lth_licd_hosp_beds_21,lth_licd_hosp_beds_20,hosp_licd_nh_beds_21,hosp_licd_nh_beds_20,stgh_licd_nh_beds_21,stgh_licd_nh_beds_20,stngh_licd_nh_beds_21,stngh_licd_nh_beds_20,lth_licd_nh_beds_21,lth_licd_nh_beds_20,stgh_bassnts_21,stgh_bassnts_20,stnglth_bassnts_21,stnglth_bassnts_20,stgh_006_049_beds_21,stgh_006_049_beds_20,stgh_050_099_beds_21,stgh_050_099_beds_20,stgh_100_199_beds_21,stgh_100_199_beds_20,stgh_200_299_beds_21,stgh_200_299_beds_20,stgh_ge300_beds_21,stgh_ge300_beds_20,stnglth_006_049_beds_21,stnglth_006_049_beds_20,stnglth_050_099_beds_21,stnglth_050_099_beds_20,stnglth_100_199_beds_21,stnglth_100_199_beds_20,stnglth_200_299_beds_21,stnglth_200_299_beds_20,stnglth_ge300_beds_21,stnglth_ge300_beds_20,vetn_hosp_006_049_beds_21,vetn_hosp_006_049_beds_20,vetn_hosp_050_099_beds_21,vetn_hosp_050_099_beds_20,vetn_hosp_100_199_beds_21,vetn_hosp_100_199_beds_20,vetn_hosp_200_299_beds_21,vetn_hosp_200_299_beds_20,vetn_hosp_ge300_beds_21,vetn_hosp_ge300_beds_20,stgh_gen_med_surg_care_adult_21,stgh_gen_med_surg_care_ped_21,stgh_obstetrc_care_21,stgh_med_surg_icu_21,stgh_card_icu_21,stgh_neonat_icu_21,stgh_neonat_interm_care_21,stgh_ped_icu_21,stgh_burn_care_21,stgh_oth_specl_care_21,stgh_oth_icu_21,stgh_rehab_care_21,stgh_substnc_use_inpat_care_21,stgh_psych_care_21,stgh_skilld_nursng_care_21,stgh_interm_care_21,stgh_acute_long_t

In [28]:
ahrf_hf_sub = ['fips_st_cnty',
               'snf_22',
               'snf_beds_22',
               'snf_cert_beds_22',                
               'nurs_fac_22',                     
               'nurs_fac_beds_22',                
               'nurs_fac_cert_beds_22',         
               'home_hlth_agencs_22',
               'rural_hlth_clincs_22',            
               'ambultry_surg_ctr_22',            
               'hospcs_22',                       
               'comn_mentl_hlth_ctr_22',          
               'nhsc_activ_sites_22']


In [45]:
df3

,fips,county,state,urban,analysis_value,Primary care physicians: Primary Care Physicians Rate,Primary care physicians: Primary Care Physicians Ratio,Dentists: Dentist Rate,Dentists: Dentist Ratio,Mental health providers: Mental Health Provider Rate,Mental health providers: Mental Health Provider Ratio,Dental Health: HPSA Score,Dental Health: % of Population Below 100% Poverty,Dental Health: HPSA Estimated Underserved Population,Dental Health: HPSA Estimated Served Population,Mental Health: HPSA Score,Mental Health: % of Population Below 100% Poverty,Mental Health: HPSA Estimated Underserved Population,Mental Health: HPSA Estimated Served Population,Primary Care: HPSA Score,Primary Care: % of Population Below 100% Poverty,Primary Care: HPSA Estimated Underserved Population,Primary Care: HPSA Estimated Served Population
0,01001,Autauga County,ALABAMA,Urban,588,44.74753,2235.0,33.84095,2955.0,37.40315,2674.0,17.00,16.90,65590.0,9556.0,13.000000,18.050000,68234.000000,181525.000000,13.000000,19.126667,42831.866667,40162.333333
1,01003,Baldwin County,ALABAMA,Rural,608,68.98591,1450.0,48.84708,2047.0,99.43869,1006.0,8.50,17.90,182060.0,49960.0,13.250000,12.633333,26996.545455,15288.636364,13.666667,15.011538,16227.230769,19090.192308
2,01005,Barbour County,ALABAMA,Rural,747,36.45791,2743.0,36.60173,2732.0,24.40115,4098.0,16.25,24.40,11858.5,3260.0,14.200000,19.400000,41980.000000,112400.000000,10.888889,29.900000,-139.000000,10305.833333
3,01007,Bibb County,ALABAMA,Urban,593,58.05126,1723.0,27.10517,3689.0,27.10517,3689.0,15.80,18.15,32300.5,12598.5,16.800000,18.500000,86303.500000,3975.000000,16.750000,18.100000,2607.000000,18025.000000
4,01009,Blount County,ALABAMA,Urban,601,22.48124,4448.0,19.00517,5262.0,17.27742,5788.0,15.50,15.45,17867.0,3586.5,18.500000,16.850000,234168.500000,48520.000000,7.000000,13.500000,4170.000000,52938.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3213,72145,Vega Baja Municipio,PUERTO RICO,Urban,436,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,38534.0,48000.0,10.000000,51.800000,233700.000000,8600.000000,0.000000,NaN,43034.000000,43500.000000
3214,72147,Vieques Municipio,PUERTO RICO,Rural,304,NaN,NaN,NaN,NaN,NaN,NaN,13.00,52.60,24057.0,70120.0,12.333333,47.050000,8069.000000,0.000000,11.500000,52.600000,23926.000000,69520.000000
3215,72149,Villalba Municipio,PUERTO RICO,Urban,392,NaN,NaN,NaN,NaN,NaN,NaN,9.00,46.40,31127.5,50256.0,13.000000,48.550000,50348.500000,11300.000000,4.666667,73.900000,36449.666667,50707.666667
3216,72151,Yabucoa Municipio,PUERTO RICO,Urban,460,NaN,NaN,NaN,NaN,NaN,NaN,12.00,51.60,31808.0,68491.0,10.333333,52.550000,55490.500000,10000.000000,6.000000,100.000000,86040.666667,27000.000000


In [29]:
df4 = df3.merge(AHRF_HF[ahrf_hf_sub],how='left',left_on='fips',right_on='fips_st_cnty')
df4.drop(labels=['fips_st_cnty'],axis=1,inplace=True)

In [30]:
df4

,fips,county,state,urban,analysis_value,Primary care physicians: Primary Care Physicians Rate,Primary care physicians: Primary Care Physicians Ratio,Dentists: Dentist Rate,Dentists: Dentist Ratio,Mental health providers: Mental Health Provider Rate,Mental health providers: Mental Health Provider Ratio,Dental Health: HPSA Score,Dental Health: % of Population Below 100% Poverty,Dental Health: HPSA Estimated Underserved Population,Dental Health: HPSA Estimated Served Population,Mental Health: HPSA Score,Mental Health: % of Population Below 100% Poverty,Mental Health: HPSA Estimated Underserved Population,Mental Health: HPSA Estimated Served Population,Primary Care: HPSA Score,Primary Care: % of Population Below 100% Poverty,Primary Care: HPSA Estimated Underserved Population,Primary Care: HPSA Estimated Served Population,snf_22,snf_beds_22,snf_cert_beds_22,nurs_fac_22,nurs_fac_beds_22,nurs_fac_cert_beds_22,home_hlth_agencs_22,rural_hlth_clincs_22,ambultry_surg_ctr_22,hospcs_22,comn_mentl_hlth_ctr_22,nhsc_activ_sites_22
0,01001,Autauga County,ALABAMA,Urban,588,44.74753,2235.0,33.84095,2955.0,37.40315,2674.0,17.0,16.9,65590.0,9556.0,13.000000,18.050000,68234.000000,181525.000000,13.000000,19.126667,42831.866667,40162.333333,1.0,162.0,162.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
1,01003,Baldwin County,ALABAMA,Rural,608,68.98591,1450.0,48.84708,2047.0,99.43869,1006.0,8.5,17.9,182060.0,49960.0,13.250000,12.633333,26996.545455,15288.636364,13.666667,15.011538,16227.230769,19090.192308,7.0,772.0,772.0,0.0,0.0,0.0,6.0,0.0,1.0,2.0,2.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3216,72151,Yabucoa Municipio,PUERTO RICO,Urban,460,NaN,NaN,NaN,NaN,NaN,NaN,12.0,51.6,31808.0,68491.0,10.333333,52.550000,55490.500000,10000.000000,6.000000,100.000000,86040.666667,27000.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3217,72153,Yauco Municipio,PUERTO RICO,Urban,528,NaN,NaN,NaN,NaN,NaN,NaN,12.0,48.8,65217.5,15888.0,10.500000,49.000000,29424.000000,0.000000,4.250000,76.350000,58350.750000,30495.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [31]:
AHRF_HP = pd.read_csv('HRSA/AHRF/ahrf2023HP.csv')
AHRF_HP.fips_st_cnty = AHRF_HP.fips_st_cnty.apply(str).str.zfill(5)
ahrf_hp_sub = ['fips_st_cnty',
               'pa_npi_22',
               'aprn_npi_22',
               'np_npi_22',
               'clin_nurse_spec_npi_22',
               'crna_npi_22',
               'apn_midwvs_npi_22',
               'chiro_npi_22',
               'opto_npi_22',
               'podtrst_npi_22']
AHRF_HP[ahrf_hp_sub].isna().sum()

fips_st_cnty      0
pa_npi_22         6
                 ..
opto_npi_22       6
podtrst_npi_22    6
Length: 10, dtype: int64

In [35]:
df5 = df4.merge(AHRF_HP[ahrf_hp_sub],how='left',left_on='fips',right_on='fips_st_cnty')
df5.drop(labels=['fips_st_cnty'],axis=1,inplace=True)
df5

,fips,county,state,urban,analysis_value,Primary care physicians: Primary Care Physicians Rate,Primary care physicians: Primary Care Physicians Ratio,Dentists: Dentist Rate,Dentists: Dentist Ratio,Mental health providers: Mental Health Provider Rate,Mental health providers: Mental Health Provider Ratio,Dental Health: HPSA Score,Dental Health: % of Population Below 100% Poverty,Dental Health: HPSA Estimated Underserved Population,Dental Health: HPSA Estimated Served Population,Mental Health: HPSA Score,Mental Health: % of Population Below 100% Poverty,Mental Health: HPSA Estimated Underserved Population,Mental Health: HPSA Estimated Served Population,Primary Care: HPSA Score,Primary Care: % of Population Below 100% Poverty,Primary Care: HPSA Estimated Underserved Population,Primary Care: HPSA Estimated Served Population,snf_22,snf_beds_22,snf_cert_beds_22,nurs_fac_22,nurs_fac_beds_22,nurs_fac_cert_beds_22,home_hlth_agencs_22,rural_hlth_clincs_22,ambultry_surg_ctr_22,hospcs_22,comn_mentl_hlth_ctr_22,nhsc_activ_sites_22,pa_npi_22,aprn_npi_22,np_npi_22,clin_nurse_spec_npi_22,crna_npi_22,apn_midwvs_npi_22,chiro_npi_22,opto_npi_22,podtrst_npi_22
0,01001,Autauga County,ALABAMA,Urban,588,44.74753,2235.0,33.84095,2955.0,37.40315,2674.0,17.0,16.9,65590.0,9556.0,13.000000,18.050000,68234.000000,181525.000000,13.000000,19.126667,42831.866667,40162.333333,1.0,162.0,162.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,5.0,34.0,28.0,1.0,5.0,0.0,7.0,5.0,3.0
1,01003,Baldwin County,ALABAMA,Rural,608,68.98591,1450.0,48.84708,2047.0,99.43869,1006.0,8.5,17.9,182060.0,49960.0,13.250000,12.633333,26996.545455,15288.636364,13.666667,15.011538,16227.230769,19090.192308,7.0,772.0,772.0,0.0,0.0,0.0,6.0,0.0,1.0,2.0,2.0,6.0,41.0,229.0,195.0,3.0,30.0,1.0,70.0,32.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3216,72151,Yabucoa Municipio,PUERTO RICO,Urban,460,NaN,NaN,NaN,NaN,NaN,NaN,12.0,51.6,31808.0,68491.0,10.333333,52.550000,55490.500000,10000.000000,6.000000,100.000000,86040.666667,27000.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3217,72153,Yauco Municipio,PUERTO RICO,Urban,528,NaN,NaN,NaN,NaN,NaN,NaN,12.0,48.8,65217.5,15888.0,10.500000,49.000000,29424.000000,0.000000,4.250000,76.350000,58350.750000,30495.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,1.0,0.0,1.0,0.0,4.0,7.0,0.0


In [36]:
df5.to_csv('prelim_df.csv')

In [37]:
df5[df5['county']=='Adjuntas Municipio']

,fips,county,state,urban,analysis_value,Primary care physicians: Primary Care Physicians Rate,Primary care physicians: Primary Care Physicians Ratio,Dentists: Dentist Rate,Dentists: Dentist Ratio,Mental health providers: Mental Health Provider Rate,Mental health providers: Mental Health Provider Ratio,Dental Health: HPSA Score,Dental Health: % of Population Below 100% Poverty,Dental Health: HPSA Estimated Underserved Population,Dental Health: HPSA Estimated Served Population,Mental Health: HPSA Score,Mental Health: % of Population Below 100% Poverty,Mental Health: HPSA Estimated Underserved Population,Mental Health: HPSA Estimated Served Population,Primary Care: HPSA Score,Primary Care: % of Population Below 100% Poverty,Primary Care: HPSA Estimated Underserved Population,Primary Care: HPSA Estimated Served Population,snf_22,snf_beds_22,snf_cert_beds_22,nurs_fac_22,nurs_fac_beds_22,nurs_fac_cert_beds_22,home_hlth_agencs_22,rural_hlth_clincs_22,ambultry_surg_ctr_22,hospcs_22,comn_mentl_hlth_ctr_22,nhsc_activ_sites_22,pa_npi_22,aprn_npi_22,np_npi_22,clin_nurse_spec_npi_22,crna_npi_22,apn_midwvs_npi_22,chiro_npi_22,opto_npi_22,podtrst_npi_22
3142,72001,Adjuntas Municipio,PUERTO RICO,Rural,270,NaN,NaN,NaN,NaN,NaN,NaN,17.0,64.7,43682.5,56918.5,15.0,57.45,43855.0,38500.0,6.2,38.125,42180.0,2632.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [87]:
med_df = pd.read_csv('prelim_df.csv')

In [49]:
med_df

,Unnamed: 0,fips,county,state,urban,analysis_value,Primary care physicians: Primary Care Physicians Rate,Primary care physicians: Primary Care Physicians Ratio,Dentists: Dentist Rate,Dentists: Dentist Ratio,Mental health providers: Mental Health Provider Rate,Mental health providers: Mental Health Provider Ratio,Dental Health: HPSA Score,Dental Health: % of Population Below 100% Poverty,Dental Health: HPSA Estimated Underserved Population,Dental Health: HPSA Estimated Served Population,Mental Health: HPSA Score,Mental Health: % of Population Below 100% Poverty,Mental Health: HPSA Estimated Underserved Population,Mental Health: HPSA Estimated Served Population,Primary Care: HPSA Score,Primary Care: % of Population Below 100% Poverty,Primary Care: HPSA Estimated Underserved Population,Primary Care: HPSA Estimated Served Population,snf_22,snf_beds_22,snf_cert_beds_22,nurs_fac_22,nurs_fac_beds_22,nurs_fac_cert_beds_22,home_hlth_agencs_22,rural_hlth_clincs_22,ambultry_surg_ctr_22,hospcs_22,comn_mentl_hlth_ctr_22,nhsc_activ_sites_22,pa_npi_22,aprn_npi_22,np_npi_22,clin_nurse_spec_npi_22,crna_npi_22,apn_midwvs_npi_22,chiro_npi_22,opto_npi_22,podtrst_npi_22
0,0,1001,Autauga County,ALABAMA,Urban,588,44.74753,2235.0,33.84095,2955.0,37.40315,2674.0,17.00,16.90,65590.0,9556.0,13.000000,18.050000,68234.000000,181525.000000,13.000000,19.126667,42831.866667,40162.333333,1.0,162.0,162.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,5.0,34.0,28.0,1.0,5.0,0.0,7.0,5.0,3.0
1,1,1003,Baldwin County,ALABAMA,Rural,608,68.98591,1450.0,48.84708,2047.0,99.43869,1006.0,8.50,17.90,182060.0,49960.0,13.250000,12.633333,26996.545455,15288.636364,13.666667,15.011538,16227.230769,19090.192308,7.0,772.0,772.0,0.0,0.0,0.0,6.0,0.0,1.0,2.0,2.0,6.0,41.0,229.0,195.0,3.0,30.0,1.0,70.0,32.0,8.0
2,2,1005,Barbour County,ALABAMA,Rural,747,36.45791,2743.0,36.60173,2732.0,24.40115,4098.0,16.25,24.40,11858.5,3260.0,14.200000,19.400000,41980.000000,112400.000000,10.888889,29.900000,-139.000000,10305.833333,1.0,180.0,180.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,4.0,1.0,17.0,17.0,0.0,0.0,0.0,1.0,3.0,0.0
3,3,1007,Bibb County,ALABAMA,Urban,593,58.05126,1723.0,27.10517,3689.0,27.10517,3689.0,15.80,18.15,32300.5,12598.5,16.800000,18.500000,86303.500000,3975.000000,16.750000,18.100000,2607.000000,18025.000000,1.0,131.0,131.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,7.0,5.0,32.0,31.0,0.0,1.0,0.0,3.0,1.0,0.0
4,4,1009,Blount County,ALABAMA,Urban,601,22.48124,4448.0,19.00517,5262.0,17.27742,5788.0,15.50,15.45,17867.0,3586.5,18.500000,16.850000,234168.500000,48520.000000,7.000000,13.500000,4170.000000,52938.000000,2.0,223.0,223.0,0.0,0.0,0.0,3.0,6.0,0.0,0.0,0.0,2.0,0.0,22.0,20.0,0.0,2.0,0.0,2.0,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3213,3213,72145,Vega Baja Municipio,PUERTO RICO,Urban,436,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,38534.0,48000.0,10.000000,51.800000,233700.000000,8600.000000,0.000000,NaN,43034.000000,43500.000000,1.0,45.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,3.0,3.0,0.0,0.0,3.0,0.0,3.0,8.0,1.0
3214,3214,72147,Vieques Municipio,PUERTO RICO,Rural,304,NaN,NaN,NaN,NaN,NaN,NaN,13.00,52.60,24057.0,70120.0,12.333333,47.050000,8069.000000,0.000000,11.500000,52.600000,23926.000000,69520.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3215,3215,72149,Villalba Municipio,PUERTO RICO,Urban,392,NaN,NaN,NaN,NaN,NaN,NaN,9.00,46.40,31127.5,50256.0,13.000000,48.550000,50348.500000,11300.000000,4.666667,73.900000,36449.666667,50707.666667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0
3216,3216,72151,Yabucoa Municipio,PUERTO RICO,Urban,460,NaN,NaN,NaN,NaN,NaN,NaN,12.00,51.60,31808.0,68491.0,10.333333,52.550000,55490.500000,10000.000000,6.000000,100.000000,86040.666667,27000.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [50]:
med_df

,Unnamed: 0,fips,county,state,urban,analysis_value,Primary care physicians: Primary Care Physicians Rate,Primary care physicians: Primary Care Physicians Ratio,Dentists: Dentist Rate,Dentists: Dentist Ratio,Mental health providers: Mental Health Provider Rate,Mental health providers: Mental Health Provider Ratio,Dental Health: HPSA Score,Dental Health: % of Population Below 100% Poverty,Dental Health: HPSA Estimated Underserved Population,Dental Health: HPSA Estimated Served Population,Mental Health: HPSA Score,Mental Health: % of Population Below 100% Poverty,Mental Health: HPSA Estimated Underserved Population,Mental Health: HPSA Estimated Served Population,Primary Care: HPSA Score,Primary Care: % of Population Below 100% Poverty,Primary Care: HPSA Estimated Underserved Population,Primary Care: HPSA Estimated Served Population,snf_22,snf_beds_22,snf_cert_beds_22,nurs_fac_22,nurs_fac_beds_22,nurs_fac_cert_beds_22,home_hlth_agencs_22,rural_hlth_clincs_22,ambultry_surg_ctr_22,hospcs_22,comn_mentl_hlth_ctr_22,nhsc_activ_sites_22,pa_npi_22,aprn_npi_22,np_npi_22,clin_nurse_spec_npi_22,crna_npi_22,apn_midwvs_npi_22,chiro_npi_22,opto_npi_22,podtrst_npi_22
0,0,1001,Autauga County,ALABAMA,Urban,588,44.74753,2235.0,33.84095,2955.0,37.40315,2674.0,17.00,16.90,65590.0,9556.0,13.000000,18.050000,68234.000000,181525.000000,13.000000,19.126667,42831.866667,40162.333333,1.0,162.0,162.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,5.0,34.0,28.0,1.0,5.0,0.0,7.0,5.0,3.0
1,1,1003,Baldwin County,ALABAMA,Rural,608,68.98591,1450.0,48.84708,2047.0,99.43869,1006.0,8.50,17.90,182060.0,49960.0,13.250000,12.633333,26996.545455,15288.636364,13.666667,15.011538,16227.230769,19090.192308,7.0,772.0,772.0,0.0,0.0,0.0,6.0,0.0,1.0,2.0,2.0,6.0,41.0,229.0,195.0,3.0,30.0,1.0,70.0,32.0,8.0
2,2,1005,Barbour County,ALABAMA,Rural,747,36.45791,2743.0,36.60173,2732.0,24.40115,4098.0,16.25,24.40,11858.5,3260.0,14.200000,19.400000,41980.000000,112400.000000,10.888889,29.900000,-139.000000,10305.833333,1.0,180.0,180.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,4.0,1.0,17.0,17.0,0.0,0.0,0.0,1.0,3.0,0.0
3,3,1007,Bibb County,ALABAMA,Urban,593,58.05126,1723.0,27.10517,3689.0,27.10517,3689.0,15.80,18.15,32300.5,12598.5,16.800000,18.500000,86303.500000,3975.000000,16.750000,18.100000,2607.000000,18025.000000,1.0,131.0,131.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,7.0,5.0,32.0,31.0,0.0,1.0,0.0,3.0,1.0,0.0
4,4,1009,Blount County,ALABAMA,Urban,601,22.48124,4448.0,19.00517,5262.0,17.27742,5788.0,15.50,15.45,17867.0,3586.5,18.500000,16.850000,234168.500000,48520.000000,7.000000,13.500000,4170.000000,52938.000000,2.0,223.0,223.0,0.0,0.0,0.0,3.0,6.0,0.0,0.0,0.0,2.0,0.0,22.0,20.0,0.0,2.0,0.0,2.0,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3213,3213,72145,Vega Baja Municipio,PUERTO RICO,Urban,436,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,38534.0,48000.0,10.000000,51.800000,233700.000000,8600.000000,0.000000,NaN,43034.000000,43500.000000,1.0,45.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,3.0,3.0,0.0,0.0,3.0,0.0,3.0,8.0,1.0
3214,3214,72147,Vieques Municipio,PUERTO RICO,Rural,304,NaN,NaN,NaN,NaN,NaN,NaN,13.00,52.60,24057.0,70120.0,12.333333,47.050000,8069.000000,0.000000,11.500000,52.600000,23926.000000,69520.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3215,3215,72149,Villalba Municipio,PUERTO RICO,Urban,392,NaN,NaN,NaN,NaN,NaN,NaN,9.00,46.40,31127.5,50256.0,13.000000,48.550000,50348.500000,11300.000000,4.666667,73.900000,36449.666667,50707.666667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0
3216,3216,72151,Yabucoa Municipio,PUERTO RICO,Urban,460,NaN,NaN,NaN,NaN,NaN,NaN,12.00,51.60,31808.0,68491.0,10.333333,52.550000,55490.500000,10000.000000,6.000000,100.000000,86040.666667,27000.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [55]:
med_df[med_df['Primary care physicians: Primary Care Physicians Rate'].isna()].state.value_counts()
for i in med_df.state.unique():
    med_df[med_df['state']==i]['Primary care physicians: Primary Care Physicians Rate'].mean()

PUERTO RICO       76
TEXAS             21
GEORGIA           16
NORTH DAKOTA      14
SOUTH DAKOTA      12
KANSAS            11
NEBRASKA           9
ILLINOIS           8
VIRGINIA           6
OKLAHOMA           6
COLORADO           5
TENNESSEE          5
KENTUCKY           5
MISSISSIPPI        4
MONTANA            4
MISSOURI           4
MINNESOTA          4
FLORIDA            3
WASHINGTON         3
INDIANA            3
NEVADA             2
NORTH CAROLINA     2
PENNSYLVANIA       2
ARKANSAS           2
MICHIGAN           2
ALABAMA            2
NEW MEXICO         1
LOUISIANA          1
OHIO               1
ALASKA             1
SOUTH CAROLINA     1
IDAHO              1
MARYLAND           1
CALIFORNIA         1
UTAH               1
Name: state, dtype: int64

45.21575938461539

111.33892375

52.32140666666667

46.66255547945207

...

66.7213072972973

54.30231018181817

63.09193527777776

68.99391391304349

nan

In [88]:
med_df.drop(med_df[med_df['state']=='PUERTO RICO'].index,axis=0,inplace=True)
med_df.drop('Unnamed: 0',axis=1,inplace=True)

In [65]:
med_df

,fips,county,state,urban,analysis_value,Primary care physicians: Primary Care Physicians Rate,Primary care physicians: Primary Care Physicians Ratio,Dentists: Dentist Rate,Dentists: Dentist Ratio,Mental health providers: Mental Health Provider Rate,Mental health providers: Mental Health Provider Ratio,Dental Health: HPSA Score,Dental Health: % of Population Below 100% Poverty,Dental Health: HPSA Estimated Underserved Population,Dental Health: HPSA Estimated Served Population,Mental Health: HPSA Score,Mental Health: % of Population Below 100% Poverty,Mental Health: HPSA Estimated Underserved Population,Mental Health: HPSA Estimated Served Population,Primary Care: HPSA Score,Primary Care: % of Population Below 100% Poverty,Primary Care: HPSA Estimated Underserved Population,Primary Care: HPSA Estimated Served Population,snf_22,snf_beds_22,snf_cert_beds_22,nurs_fac_22,nurs_fac_beds_22,nurs_fac_cert_beds_22,home_hlth_agencs_22,rural_hlth_clincs_22,ambultry_surg_ctr_22,hospcs_22,comn_mentl_hlth_ctr_22,nhsc_activ_sites_22,pa_npi_22,aprn_npi_22,np_npi_22,clin_nurse_spec_npi_22,crna_npi_22,apn_midwvs_npi_22,chiro_npi_22,opto_npi_22,podtrst_npi_22
0,1001,Autauga County,ALABAMA,Urban,588,44.74753,2235.0,33.84095,2955.0,37.40315,2674.0,17.00,16.90,65590.0,9556.0,13.000000,18.050000,68234.000000,181525.000000,13.000000,19.126667,42831.866667,40162.333333,1.0,162.0,162.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,5.0,34.0,28.0,1.0,5.0,0.0,7.0,5.0,3.0
1,1003,Baldwin County,ALABAMA,Rural,608,68.98591,1450.0,48.84708,2047.0,99.43869,1006.0,8.50,17.90,182060.0,49960.0,13.250000,12.633333,26996.545455,15288.636364,13.666667,15.011538,16227.230769,19090.192308,7.0,772.0,772.0,0.0,0.0,0.0,6.0,0.0,1.0,2.0,2.0,6.0,41.0,229.0,195.0,3.0,30.0,1.0,70.0,32.0,8.0
2,1005,Barbour County,ALABAMA,Rural,747,36.45791,2743.0,36.60173,2732.0,24.40115,4098.0,16.25,24.40,11858.5,3260.0,14.200000,19.400000,41980.000000,112400.000000,10.888889,29.900000,-139.000000,10305.833333,1.0,180.0,180.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,4.0,1.0,17.0,17.0,0.0,0.0,0.0,1.0,3.0,0.0
3,1007,Bibb County,ALABAMA,Urban,593,58.05126,1723.0,27.10517,3689.0,27.10517,3689.0,15.80,18.15,32300.5,12598.5,16.800000,18.500000,86303.500000,3975.000000,16.750000,18.100000,2607.000000,18025.000000,1.0,131.0,131.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,7.0,5.0,32.0,31.0,0.0,1.0,0.0,3.0,1.0,0.0
4,1009,Blount County,ALABAMA,Urban,601,22.48124,4448.0,19.00517,5262.0,17.27742,5788.0,15.50,15.45,17867.0,3586.5,18.500000,16.850000,234168.500000,48520.000000,7.000000,13.500000,4170.000000,52938.000000,2.0,223.0,223.0,0.0,0.0,0.0,3.0,6.0,0.0,0.0,0.0,2.0,0.0,22.0,20.0,0.0,2.0,0.0,2.0,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,56037,Sweetwater County,WYOMING,Rural,510,44.87164,2229.0,58.58505,1707.0,246.05723,406.0,8.50,NaN,NaN,NaN,10.250000,9.700000,26073.500000,50000.000000,6.833333,7.850000,5474.250000,25160.500000,2.0,141.0,141.0,0.0,0.0,0.0,2.0,1.0,1.0,1.0,0.0,5.0,21.0,27.0,23.0,2.0,0.0,2.0,15.0,13.0,3.0
3138,56039,Teton County,WYOMING,Rural,358,115.06989,869.0,93.62897,1068.0,544.75039,184.0,NaN,NaN,NaN,NaN,5.500000,9.700000,10999.500000,56000.000000,NaN,NaN,NaN,NaN,1.0,60.0,60.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,3.0,16.0,45.0,27.0,1.0,15.0,2.0,11.0,2.0,0.0
3139,56041,Uinta County,WYOMING,Rural,538,44.49718,2247.0,84.09597,1189.0,366.06480,273.0,5.00,NaN,NaN,NaN,9.750000,9.700000,17516.500000,50000.000000,8.714286,10.200000,-2511.800000,6890.000000,1.0,60.0,60.0,0.0,0.0,0.0,3.0,3.0,1.0,2.0,0.0,7.0,7.0,20.0,12.0,4.0,4.0,0.0,6.0,7.0,0.0
3140,56043,Washakie County,WYOMING,Rural,665,64.06150,1561.0,90.20619,1109.0,399.48454,250.0,5.00,NaN,NaN,NaN,13.666667,10.200000,22478.000000,30000.000000,8.250000,NaN,1345.500000,3850.000000,1.0,87.0,87.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,4.0,7.0,5.0,0.0,2.0,0.0,3.0,6.0,1.0


In [66]:
med_df[med_df['Primary care physicians: Primary Care Physicians Rate'].isna()].state.value_counts()
for i in med_df.state.unique():
    med_df[med_df['state']==i]['Primary care physicians: Primary Care Physicians Rate'].mean()

TEXAS             21
GEORGIA           16
NORTH DAKOTA      14
SOUTH DAKOTA      12
KANSAS            11
NEBRASKA           9
ILLINOIS           8
VIRGINIA           6
OKLAHOMA           6
COLORADO           5
TENNESSEE          5
KENTUCKY           5
MISSOURI           4
MISSISSIPPI        4
MONTANA            4
MINNESOTA          4
WASHINGTON         3
INDIANA            3
FLORIDA            3
MICHIGAN           2
ARKANSAS           2
PENNSYLVANIA       2
ALABAMA            2
NORTH CAROLINA     2
NEVADA             2
LOUISIANA          1
OHIO               1
NEW MEXICO         1
SOUTH CAROLINA     1
IDAHO              1
CALIFORNIA         1
UTAH               1
MARYLAND           1
ALASKA             1
Name: state, dtype: int64

45.21575938461539

111.33892375

52.32140666666667

46.66255547945207

...

60.40164330708664

66.7213072972973

54.30231018181817

63.09193527777776

68.99391391304349

In [126]:
def fill_missing(column,df):
    df[column] = df.apply(lambda row: df[df['state']==row['state']][column].mean() if np.isnan(row[column]) else row[column],axis=1)
med_copy = med_df.copy()
for column in med_copy.select_dtypes(include='number'):
    fill_missing(column,med_copy)
med_copy.isna().sum()

fips                                                       0
county                                                    13
state                                                      0
urban                                                      0
analysis_value                                             0
Primary care physicians: Primary Care Physicians Rate      0
Primary care physicians: Primary Care Physicians Ratio     0
Dentists: Dentist Rate                                     0
Dentists: Dentist Ratio                                    0
Mental health providers: Mental Health Provider Rate       0
Mental health providers: Mental Health Provider Ratio      0
Dental Health: HPSA Score                                  0
Dental Health: % of Population Below 100% Poverty          0
Dental Health: HPSA Estimated Underserved Population       0
Dental Health: HPSA Estimated Served Population            0
Mental Health: HPSA Score                                  0
Mental Health: % of Popu

In [117]:
med_copy[med_copy['Mental Health: HPSA Estimated Underserved Population'].isna()]

,fips,county,state,urban,analysis_value,Primary care physicians: Primary Care Physicians Rate,Primary care physicians: Primary Care Physicians Ratio,Dentists: Dentist Rate,Dentists: Dentist Ratio,Mental health providers: Mental Health Provider Rate,Mental health providers: Mental Health Provider Ratio,Dental Health: HPSA Score,Dental Health: % of Population Below 100% Poverty,Dental Health: HPSA Estimated Underserved Population,Dental Health: HPSA Estimated Served Population,Mental Health: HPSA Score,Mental Health: % of Population Below 100% Poverty,Mental Health: HPSA Estimated Underserved Population,Mental Health: HPSA Estimated Served Population,Primary Care: HPSA Score,Primary Care: % of Population Below 100% Poverty,Primary Care: HPSA Estimated Underserved Population,Primary Care: HPSA Estimated Served Population,snf_22,snf_beds_22,snf_cert_beds_22,nurs_fac_22,nurs_fac_beds_22,nurs_fac_cert_beds_22,home_hlth_agencs_22,rural_hlth_clincs_22,ambultry_surg_ctr_22,hospcs_22,comn_mentl_hlth_ctr_22,nhsc_activ_sites_22,pa_npi_22,aprn_npi_22,np_npi_22,clin_nurse_spec_npi_22,crna_npi_22,apn_midwvs_npi_22,chiro_npi_22,opto_npi_22,podtrst_npi_22
2805,50001,Addison County,VERMONT,Rural,739,114.20181,876.0,51.55898,1940.0,317.49478,315.0,0.428571,0.000000,-3690.769231,4600.000000,14.000000,0.0,NaN,NaN,0.625000,12.500000,609.285714,7850.000000,1.0,98.0,98.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,10.0,46.0,34.0,1.0,6.0,5.0,13.0,9.0,1.0
2806,50003,Bennington County,VERMONT,Rural,645,101.49422,985.0,90.55408,1104.0,495.21761,202.0,6.555556,13.300000,1262.142857,5535.714286,10.000000,0.0,NaN,NaN,3.222222,5.514286,2220.142857,1542.857143,4.0,517.0,517.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,14.0,47.0,40.0,4.0,0.0,3.0,10.0,4.0,7.0
2807,50005,Caledonia County,VERMONT,Rural,625,83.35278,1200.0,74.06161,1350.0,393.87309,254.0,5.285714,9.631429,5166.000000,4400.000000,13.600000,0.0,NaN,NaN,1.176471,10.880000,505.307692,7000.000000,2.0,170.0,170.0,0.0,0.0,0.0,1.0,4.0,0.0,1.0,0.0,6.0,11.0,55.0,45.0,2.0,4.0,4.0,11.0,9.0,2.0
2808,50007,Chittenden County,VERMONT,Urban,567,186.84284,535.0,97.37928,1027.0,698.08771,143.0,13.000000,9.631429,2021.593004,5078.683261,21.000000,0.0,NaN,NaN,9.000000,9.464389,1889.655244,5333.164689,5.0,526.0,526.0,0.0,0.0,0.0,2.0,0.0,2.0,1.0,0.0,10.0,141.0,289.0,257.0,4.0,10.0,18.0,93.0,43.0,5.0
2809,50009,Essex County,VERMONT,Rural,613,16.22586,6163.0,48.99559,2041.0,65.32745,1531.0,0.000000,9.631429,1600.000000,5000.000000,0.000000,0.0,NaN,NaN,2.080000,12.450000,-334.680000,4003.760000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
2810,50011,Franklin County,VERMONT,Urban,754,54.65366,1830.0,48.30432,2070.0,259.63570,385.0,16.500000,9.631429,2021.593004,5078.683261,12.000000,0.0,NaN,NaN,1.857143,9.464389,3033.000000,7350.000000,3.0,209.0,209.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,9.0,31.0,33.0,32.0,0.0,0.0,1.0,7.0,4.0,1.0
2811,50013,Grand Isle County,VERMONT,Urban,489,69.10850,1447.0,13.94895,7169.0,27.89789,3585.0,0.000000,11.500000,1564.000000,5500.000000,0.000000,0.0,NaN,NaN,0.000000,12.000000,1600.000000,3500.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
2812,50015,Lamoille County,VERMONT,Rural,578,86.74395,1153.0,51.30026,1949.0,520.89499,192.0,1.923077,9.631429,5166.000000,4400.000000,12.000000,0.0,NaN,NaN,2.000000,7.200000,12550.500000,3850.000000,1.0,72.0,72.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,4.0,10.0,42.0,32.0,0.0,4.0,6.0,9.0,4.0,0.0
2813,50017,Orange County,VERMONT,Rural,594,89.99031,1111.0,31.20990,3204.0,357.18001,280.0,3.428571,9.631429,2641.000000,2400.000000,10.333333,0.0,NaN,NaN,8.071429,11.550000,1779.000000,6391.000000,1.0,30.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,11.0,32.0,24.0,0.0,0.0,8.0,6.0,3.0,3.0
2814,50019,Orleans County,VERMONT,Rural,683,70.27407,1423.0,63.20407,1582.0,289.99517,345.0,6.565217,16.900000,6806.600000,13153.333333,7.111111,0.0,NaN,NaN,3.150943,7.178378,11130.952381,3291.857143,5.0,239.0,239.0,0.0,0.0,0.0,1.0,4.

In [131]:
mh_under_avg = med_copy['Mental Health: HPSA Estimated Underserved Population'].mean()
mh_serv_avg = med_copy['Mental Health: HPSA Estimated Served Population'].mean()
med_copy['Mental Health: HPSA Estimated Underserved Population'] = \
med_copy.apply(lambda row: mh_under_avg if np.isnan(row['Mental Health: HPSA Estimated Underserved Population'])\
               else row['Mental Health: HPSA Estimated Underserved Population'],axis=1)
med_copy['Mental Health: HPSA Estimated Served Population'] = \
med_copy.apply(lambda row: mh_serv_avg if np.isnan(row['Mental Health: HPSA Estimated Served Population'])\
               else row['Mental Health: HPSA Estimated Served Population'],axis=1)
med_copy.isna().sum()

fips                                                       0
county                                                    13
state                                                      0
urban                                                      0
analysis_value                                             0
Primary care physicians: Primary Care Physicians Rate      0
Primary care physicians: Primary Care Physicians Ratio     0
Dentists: Dentist Rate                                     0
Dentists: Dentist Ratio                                    0
Mental health providers: Mental Health Provider Rate       0
Mental health providers: Mental Health Provider Ratio      0
Dental Health: HPSA Score                                  0
Dental Health: % of Population Below 100% Poverty          0
Dental Health: HPSA Estimated Underserved Population       0
Dental Health: HPSA Estimated Served Population            0
Mental Health: HPSA Score                                  0
Mental Health: % of Popu

In [132]:
med_copy.to_csv('filled_na.csv')

In [133]:
med_encoded = pd.get_dummies(med_copy, columns=['urban', ])

In [134]:
med_encoded

,fips,county,state,analysis_value,Primary care physicians: Primary Care Physicians Rate,Primary care physicians: Primary Care Physicians Ratio,Dentists: Dentist Rate,Dentists: Dentist Ratio,Mental health providers: Mental Health Provider Rate,Mental health providers: Mental Health Provider Ratio,Dental Health: HPSA Score,Dental Health: % of Population Below 100% Poverty,Dental Health: HPSA Estimated Underserved Population,Dental Health: HPSA Estimated Served Population,Mental Health: HPSA Score,Mental Health: % of Population Below 100% Poverty,Mental Health: HPSA Estimated Underserved Population,Mental Health: HPSA Estimated Served Population,Primary Care: HPSA Score,Primary Care: % of Population Below 100% Poverty,Primary Care: HPSA Estimated Underserved Population,Primary Care: HPSA Estimated Served Population,snf_22,snf_beds_22,snf_cert_beds_22,nurs_fac_22,nurs_fac_beds_22,nurs_fac_cert_beds_22,home_hlth_agencs_22,rural_hlth_clincs_22,ambultry_surg_ctr_22,hospcs_22,comn_mentl_hlth_ctr_22,nhsc_activ_sites_22,pa_npi_22,aprn_npi_22,np_npi_22,clin_nurse_spec_npi_22,crna_npi_22,apn_midwvs_npi_22,chiro_npi_22,opto_npi_22,podtrst_npi_22,urban_Rural,urban_Urban
0,1001,Autauga County,ALABAMA,588,44.74753,2235.0,33.84095,2955.0,37.40315,2674.0,17.00000,16.90,65590.000000,9556.000000,13.000000,18.050000,68234.000000,181525.000000,13.000000,19.126667,42831.866667,40162.333333,1.0,162.0,162.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,5.0,34.0,28.0,1.0,5.0,0.0,7.0,5.0,3.0,0,1
1,1003,Baldwin County,ALABAMA,608,68.98591,1450.0,48.84708,2047.0,99.43869,1006.0,8.50000,17.90,182060.000000,49960.000000,13.250000,12.633333,26996.545455,15288.636364,13.666667,15.011538,16227.230769,19090.192308,7.0,772.0,772.0,0.0,0.0,0.0,6.0,0.0,1.0,2.0,2.0,6.0,41.0,229.0,195.0,3.0,30.0,1.0,70.0,32.0,8.0,1,0
2,1005,Barbour County,ALABAMA,747,36.45791,2743.0,36.60173,2732.0,24.40115,4098.0,16.25000,24.40,11858.500000,3260.000000,14.200000,19.400000,41980.000000,112400.000000,10.888889,29.900000,-139.000000,10305.833333,1.0,180.0,180.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,4.0,1.0,17.0,17.0,0.0,0.0,0.0,1.0,3.0,0.0,1,0
3,1007,Bibb County,ALABAMA,593,58.05126,1723.0,27.10517,3689.0,27.10517,3689.0,15.80000,18.15,32300.500000,12598.500000,16.800000,18.500000,86303.500000,3975.000000,16.750000,18.100000,2607.000000,18025.000000,1.0,131.0,131.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,7.0,5.0,32.0,31.0,0.0,1.0,0.0,3.0,1.0,0.0,0,1
4,1009,Blount County,ALABAMA,601,22.48124,4448.0,19.00517,5262.0,17.27742,5788.0,15.50000,15.45,17867.000000,3586.500000,18.500000,16.850000,234168.500000,48520.000000,7.000000,13.500000,4170.000000,52938.000000,2.0,223.0,223.0,0.0,0.0,0.0,3.0,6.0,0.0,0.0,0.0,2.0,0.0,22.0,20.0,0.0,2.0,0.0,2.0,4.0,0.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,56037,Sweetwater County,WYOMING,510,44.87164,2229.0,58.58505,1707.0,246.05723,406.0,8.50000,8.76,-725.941176,6558.647059,10.250000,9.700000,26073.500000,50000.000000,6.833333,7.850000,5474.250000,25160.500000,2.0,141.0,141.0,0.0,0.0,0.0,2.0,1.0,1.0,1.0,0.0,5.0,21.0,27.0,23.0,2.0,0.0,2.0,15.0,13.0,3.0,1,0
3138,56039,Teton County,WYOMING,358,115.06989,869.0,93.62897,1068.0,544.75039,184.0,8.80303,8.76,-725.941176,6558.647059,5.500000,9.700000,10999.500000,56000.000000,8.919850,10.623333,2357.838745,4740.009470,1.0,60.0,60.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,3.0,16.0,45.0,27.0,1.0,15.0,2.0,11.0,2.0,0.0,1,0
3139,56041,Uinta County,WYOMING,538,44.49718,2247.0,84.09597,1189.0,366.06480,273.0,5.00000,8.76,-725.941176,6558.647059,9.750000,9.700000,17516.500000,50000.000000,8.714286,10.200000,-2511.800000,6890.000000,1.0,60.0,60.0,0.0,0.0,0.0,3.0,3.0,1.0,2.0,0.0,7.0,7.0,20.0,12.0,4.0,4.0,0.0,6.0,7.0,0.0,1,0
3140,56043,Washakie County,WYOMING,665,64.06150,1561.0,90.20619,1109.0,399.48454,250.0,5.00000,8.76,-725.941176,6558.647059,13.666667,10.200000,22478.000000,30000.000000,8.250000,10.62

In [135]:
med_encoded.to_csv('ohe_med.csv')